# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [ ]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-e8291c15f6-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
       results = sparql.query()
       json_results = results.convert()
       if len(json_results['results']['bindings'])==0:
          print("Empty")
          return 0
    
       for bindings in json_results['results']['bindings']:
          print( [ (var, value['value'])  for var, value in bindings.items() ] )

       return len(json_results['results']['bindings'])

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Book Workflow Series ("Author comparison explorative search") 

Consider the following exploratory scenario:


>  Investigate Italian and French book authors in terms of awards, books published and copyright types



## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P279`    | subclass      | predicate |
| `wdt:P106`    | occupation    | predicate | 
| `wdt:P17`     | country       | predicate | 
| `wdt:P27`     | citizenship   | predicate | 
| `wd:Q36180`   | writer        | node |
| `wd:Q38`      | Italy         | node |
| `wd:Q172579`  | Kingdom of Italy        | node |
| `wd:Q142`     | France        | node |
| `wd:Q37922`   | Nobel Prize literature        | node |
| `wd:Q213678`  | Vatican Library        | node |


Also consider that

```
?p wdt:P27 wd:Q142
```

is the BGP to retrieve all **French citizens**


The workload should


1. Identify the BGP for obtaining the Italian and French writers who published a book in the last 50 years

2. Compare the number of books written by Italian and French writers

3. Count how many books written by Italian authors are now released with a "public domain" copyright form

4. How many Literature Nobel awards won authors from Italy and from the Kingdom of Italy? 

5. Are there books from Litarature Nobel Award winners which are not present in the Vatican Library? (if so, who is the author with more books not in the Vatical Library)?

In [ ]:
# start your workflow here

In [ ]:
queryString = """
SELECT COUNT(?p)
WHERE { 
?p wdt:P27 wd:Q142 .
} 
"""

print("Results")
run_query(queryString)

## Task 1 - Identify the BGP for obtaining the Italian and French writers who published a book in the last 50 years

#### I start by finding all French and Italian writers

In [ ]:
queryString = """
SELECT DISTINCT ?name WHERE { 

{?writer wdt:P27 wd:Q142;
         wdt:P106 wd:Q36180.
         }
UNION
{?writer wdt:P27 wd:Q38;
         wdt:P106 wd:Q36180.}

?writer <http://schema.org/name> ?name .

} limit 100

"""

print("Results")
run_query(queryString)

#### I inspect their properties

In [32]:
queryString = """
SELECT DISTINCT ?p ?pname (COUNT(?obj) AS ?count) WHERE { 

{?writer wdt:P27 wd:Q142;
         wdt:P106 wd:Q36180;
         ?p ?obj.
         }
UNION
{?writer wdt:P27 wd:Q38;
         wdt:P106 wd:Q36180;
         ?p ?obj.
         }

?p <http://schema.org/name> ?pname .

}GROUP BY ?p ?pname
ORDER BY DESC (?count)
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pname', 'occupation'), ('count', '53470')]
[('p', 'http://www.wikidata.org/prop/direct/P27'), ('pname', 'country of citizenship'), ('count', '24226')]
[('p', 'http://www.wikidata.org/prop/direct/P1412'), ('pname', 'languages spoken, written or signed'), ('count', '22225')]
[('p', 'http://www.wikidata.org/prop/direct/P735'), ('pname', 'given name'), ('count', '20753')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('count', '20680')]
[('p', 'http://www.wikidata.org/prop/direct/P21'), ('pname', 'sex or gender'), ('count', '20634')]
[('p', 'http://www.wikidata.org/prop/direct/P569'), ('pname', 'date of birth'), ('count', '20280')]
[('p', 'http://www.wikidata.org/prop/direct/P214'), ('pname', 'VIAF ID'), ('count', '19026')]
[('p', 'http://www.wikidata.org/prop/direct/P7859'), ('pname', 'WorldCat Identities ID'), ('count', '18025')]
[('p', 'http://www.wikidata.org/prop/direct/P19'), ('pname', 'place

100

#### Let's try with "notable work" (wdt:P800)

In [ ]:
queryString= """
SELECT DISTINCT ?work ?workname WHERE { 

{?writer wdt:P27 wd:Q142;
         wdt:P106 wd:Q36180;
         wdt:P800 ?work.
         }
UNION
{?writer wdt:P27 wd:Q38;
         wdt:P106 wd:Q36180;
         wdt:P800 ?work.
         }

?work <http://schema.org/name> ?workname.

}
"""

print("Results")
run_query(queryString)

#### I inspect the properties to find only the books

In [23]:
queryString = """
SELECT DISTINCT ?p ?pname (COUNT(?obj) AS ?count) WHERE { 

{?writer wdt:P27 wd:Q142;
         wdt:P106 wd:Q36180;
         wdt:P800 ?work.
?work ?p ?obj.
         }
UNION
{?writer wdt:P27 wd:Q38;
         wdt:P106 wd:Q36180;
         wdt:P800 ?work.
?work ?p ?obj.
         }

?p <http://schema.org/name> ?pname .

}GROUP BY ?p ?pname
ORDER BY DESC (?count)
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('count', '2290')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('pname', 'genre'), ('count', '1780')]
[('p', 'http://www.wikidata.org/prop/direct/P50'), ('pname', 'author'), ('count', '1712')]
[('p', 'http://www.wikidata.org/prop/direct/P577'), ('pname', 'publication date'), ('count', '1679')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('pname', 'cast member'), ('count', '1581')]
[('p', 'http://www.wikidata.org/prop/direct/P407'), ('pname', 'language of work or name'), ('count', '1428')]
[('p', 'http://www.wikidata.org/prop/direct/P495'), ('pname', 'country of origin'), ('count', '1280')]
[('p', 'http://www.wikidata.org/prop/direct/P2671'), ('pname', 'Google Knowledge Graph ID'), ('count', '857')]
[('p', 'http://www.wikidata.org/prop/direct/P646'), ('pname', 'Freebase ID'), ('count', '734')]
[('p', 'http://www.wikidata.org/prop/direct/P123'), ('pname', 'publisher'), ('count', '657')]
[

100

#### Search in "instance of"

In [ ]:
queryString = """
SELECT DISTINCT ?instance ?instancename (COUNT(?instancename) AS ?count) WHERE { 

{?writer wdt:P27 wd:Q142;
         wdt:P106 wd:Q36180;
         wdt:P800 ?work.
?work wdt:P31 ?instance.
         }
UNION
{?writer wdt:P27 wd:Q38;
         wdt:P106 wd:Q36180;
         wdt:P800 ?work.
?work wdt:P31 ?instance.
         }

?instance <http://schema.org/name> ?instancename.

}
GROUP BY ?instance ?instancename
ORDER BY DESC (?count)
LIMIT 50
"""

print("Results")
run_query(queryString)

#### I found "written work" (wd:Q47461344), "literary work" (wd:Q7725634), "book" (wd:Q571), "novel" (wd:Q8261)

In [ ]:
queryString = """
SELECT DISTINCT ?work ?workname WHERE { 

{?writer wdt:P27 wd:Q142;
         wdt:P106 wd:Q36180;
         wdt:P800 ?work.
?work wdt:P31 ?type.
        }
UNION

{?writer wdt:P27 wd:Q38;
         wdt:P106 wd:Q36180;
         wdt:P800 ?work.
?work wdt:P31 ?type.

         }

?work <http://schema.org/name> ?workname.

FILTER (?type=wd:Q47461344 || ?type=wd:Q7725634 || ?type=wd:Q571 || ?type=wd:Q8261)
}

"""

print("Results")
run_query(queryString)

#### I use "publication date"(wdt:P577) found before to select only work published in the last 50 years

In [18]:
queryString = """
SELECT DISTINCT ?work WHERE { 
{
?writer wdt:P27 wd:Q142;
         wdt:P106 wd:Q36180;
         wdt:P800 ?work.
?work wdt:P31 ?type;
      wdt:P577 ?date.
}
UNION

{?writer wdt:P27 wd:Q38;
         wdt:P106 wd:Q36180;
         wdt:P800 ?work.
?work wdt:P31 ?type;
      wdt:P577 ?date.
         }


FILTER ((?type=wd:Q47461344 || ?type=wd:Q7725634 || ?type=wd:Q571 || ?type=wd:Q8261) && (?date>"1972-01-01T00:00:00Z"^^xsd:dateTime))
}
GROUP BY ?work ?workname
ORDER BY DESC (?authors)

"""

print("Results")
run_query(queryString)

Results
[('work', 'http://www.wikidata.org/entity/Q3549587')]
[('work', 'http://www.wikidata.org/entity/Q3226919')]
[('work', 'http://www.wikidata.org/entity/Q2860454')]
[('work', 'http://www.wikidata.org/entity/Q16652708')]
[('work', 'http://www.wikidata.org/entity/Q106368932')]
[('work', 'http://www.wikidata.org/entity/Q3564414')]
[('work', 'http://www.wikidata.org/entity/Q18563318')]
[('work', 'http://www.wikidata.org/entity/Q3365485')]
[('work', 'http://www.wikidata.org/entity/Q3358641')]
[('work', 'http://www.wikidata.org/entity/Q18215049')]
[('work', 'http://www.wikidata.org/entity/Q40014992')]
[('work', 'http://www.wikidata.org/entity/Q2930400')]
[('work', 'http://www.wikidata.org/entity/Q2844011')]
[('work', 'http://www.wikidata.org/entity/Q3234359')]
[('work', 'http://www.wikidata.org/entity/Q3240199')]
[('work', 'http://www.wikidata.org/entity/Q668506')]
[('work', 'http://www.wikidata.org/entity/Q3201748')]
[('work', 'http://www.wikidata.org/entity/Q3211527')]
[('work', 'http

512

#### Now Select only the authors

In [17]:
queryString = """
SELECT DISTINCT ?writer ?writername WHERE { 
{
?writer wdt:P27 wd:Q142;
         wdt:P106 wd:Q36180;
         wdt:P800 ?work.
?work wdt:P31 ?type;
      wdt:P577 ?date.
}
UNION

{?writer wdt:P27 wd:Q38;
         wdt:P106 wd:Q36180;
         wdt:P800 ?work.
?work wdt:P31 ?type;
      wdt:P577 ?date.
         }

?writer <http://schema.org/name> ?writername.

FILTER ((?type=wd:Q47461344 || ?type=wd:Q7725634 || ?type=wd:Q571 || ?type=wd:Q8261) && (?date>"1972-01-01T00:00:00Z"^^xsd:dateTime))
}

"""

print("Results")
run_query(queryString)

Results
[('writer', 'http://www.wikidata.org/entity/Q435402'), ('writername', 'Sylvie Germain')]
[('writer', 'http://www.wikidata.org/entity/Q555545'), ('writername', 'Michel del Castillo')]
[('writer', 'http://www.wikidata.org/entity/Q447507'), ('writername', 'Anne Wiazemsky')]
[('writer', 'http://www.wikidata.org/entity/Q2296371'), ('writername', 'Soazig Aaron')]
[('writer', 'http://www.wikidata.org/entity/Q2411481'), ('writername', 'Patrick Deville')]
[('writer', 'http://www.wikidata.org/entity/Q29766'), ('writername', 'Jean Venturini')]
[('writer', 'http://www.wikidata.org/entity/Q520892'), ('writername', 'Bernard Clavel')]
[('writer', 'http://www.wikidata.org/entity/Q380443'), ('writername', 'Alain Finkielkraut')]
[('writer', 'http://www.wikidata.org/entity/Q3201139'), ('writername', 'Kéthévane Davrichewy')]
[('writer', 'http://www.wikidata.org/entity/Q92635'), ('writername', 'Jacques Vallée')]
[('writer', 'http://www.wikidata.org/entity/Q364315'), ('writername', 'Jacques Attali')

329

## Task 2 - Compare the number of books written by Italian and French writers

In [22]:
queryString="""
SELECT DISTINCT ?Country (COUNT(?work) as ?number) WHERE { 


?writer wdt:P27 ?nation;
         wdt:P106 wd:Q36180;
         wdt:P800 ?work.
?work wdt:P31 ?type.
      

?nation <http://schema.org/name> ?Country.
FILTER ((?nation=wd:Q142 || ?nation=wd:Q38) && (?type=wd:Q47461344 || ?type=wd:Q7725634 || ?type=wd:Q571 || ?type=wd:Q8261))
}
GROUP BY ?Country
"""
print("Results")
run_query(queryString)

Results
[('Country', 'France'), ('number', '1146')]
[('Country', 'Italy'), ('number', '175')]


2

## Task 3 - Count how many books written by Italian authors are now released with a "public domain" copyright form

#### I found "Copyright Status" (wst:P6216). I query all possible values

In [24]:
queryString= """
SELECT DISTINCT ?obj ?objname WHERE{

?writer wdt:P27 wd:Q38;
         wdt:P106 wd:Q36180;
         wdt:P800 ?work.
?work wdt:P31 ?type;
      wdt:P6216 ?obj.
    
?obj <http://schema.org/name> ?objname.

FILTER (?type=wd:Q47461344 || ?type=wd:Q7725634 || ?type=wd:Q571 || ?type=wd:Q8261)
}

"""

print("Results")
run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q19652'), ('objname', 'public domain')]


1

#### I count them

In [28]:
queryString= """
SELECT DISTINCT (COUNT(?work) AS ?number) WHERE{

?writer wdt:P27 wd:Q38;
         wdt:P106 wd:Q36180;
         wdt:P800 ?work.
?work wdt:P31 ?type;
      wdt:P6216 wd:Q19652.
    
FILTER (?type=wd:Q47461344 || ?type=wd:Q7725634 || ?type=wd:Q571 || ?type=wd:Q8261)
}
"""
print("Results")
run_query(queryString)

Results
[('number', '5')]


1

In [27]:
queryString= """
SELECT DISTINCT ?workname WHERE{

?writer wdt:P27 wd:Q38;
         wdt:P106 wd:Q36180;
         wdt:P800 ?work.
?work wdt:P31 ?type;
      wdt:P6216 wd:Q19652.

?work <http://schema.org/name> ?workname.
FILTER (?type=wd:Q47461344 || ?type=wd:Q7725634 || ?type=wd:Q571 || ?type=wd:Q8261)
}
"""
print("Results")
run_query(queryString)

Results
[('workname', 'Lives of the Most Excellent Painters, Sculptors, and Architects')]
[('workname', 'Nibelungenlied')]
[('workname', 'Captain Blood')]
[('workname', 'Scaramouche')]


4

## Task 4 - How many Literature Nobel awards won authors from Italy and from the Kingdom of Italy?

#### I search  for the properties of the "Nobel Prize in Literature" (wd:Q37922)

In [30]:
queryString = """
SELECT DISTINCT ?p ?pname WHERE { 

wd:Q37922 ?p ?obj.
?p <http://schema.org/name> ?pname.
 }
"""
print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('pname', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pname', 'winner')]
[('p', 'http://www.wikidata.org/prop/direct/P138'), ('pname', 'named after')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('pname', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P1705'), ('pname', 'native label')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('pname', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('pname', 'GND ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2354'), ('pname', 'has list')]
[('p', 'http://www.wikidata.org/prop/direct/P2581'), ('pname', 'BabelNet ID')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('pname', 'location')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/

33

#### I search for all winners (wdt:P1346)

In [31]:
queryString = """
SELECT DISTINCT ?winner ?winnername WHERE { 

wd:Q37922 wdt:P1346 ?winner.
?winner <http://schema.org/name> ?winnername.
 }
"""
print("Results")
run_query(queryString)

Results
[('winner', 'http://www.wikidata.org/entity/Q159552'), ('winnername', 'Johannes V. Jensen')]
[('winner', 'http://www.wikidata.org/entity/Q254032'), ('winnername', 'Olga Tokarczuk')]
[('winner', 'http://www.wikidata.org/entity/Q7241'), ('winnername', 'Rabindranath Tagore')]
[('winner', 'http://www.wikidata.org/entity/Q392'), ('winnername', 'Bob Dylan')]
[('winner', 'http://www.wikidata.org/entity/Q44593'), ('winnername', 'V. S. Naipaul')]
[('winner', 'http://www.wikidata.org/entity/Q121180'), ('winnername', 'Władysław Reymont')]
[('winner', 'http://www.wikidata.org/entity/Q47561'), ('winnername', 'Ivo Andrić')]
[('winner', 'http://www.wikidata.org/entity/Q6538'), ('winnername', 'Günter Grass')]
[('winner', 'http://www.wikidata.org/entity/Q44107'), ('winnername', 'Peter Handke')]
[('winner', 'http://www.wikidata.org/entity/Q72334'), ('winnername', 'Toni Morrison')]
[('winner', 'http://www.wikidata.org/entity/Q225554'), ('winnername', 'Henrik Pontoppidan')]
[('winner', 'http://www

15

#### Few Data i search the other way around starting from "award received"(wdt:P166)  from authors

In [ ]:
queryString = """
SELECT DISTINCT ?award ?awardname WHERE { 

{?writer wdt:P27 wd:Q142;
         wdt:P106 wd:Q36180;
         wdt:P166 ?award.
         }
UNION
{?writer wdt:P27 wd:Q38;
         wdt:P106 wd:Q36180;
         wdt:P166 ?award.
         }

?award <http://schema.org/name> ?awardname .

}
"""

print("Results")
run_query(queryString)

#### Lot of properties i regex only "Nobel"

In [35]:
queryString = """
SELECT DISTINCT ?award ?awardname WHERE { 

{?writer wdt:P27 wd:Q142;
         wdt:P106 wd:Q36180;
         wdt:P166 ?award.
         }
UNION
{?writer wdt:P27 wd:Q38;
         wdt:P106 wd:Q36180;
         wdt:P166 ?award.
         }

?award <http://schema.org/name> ?awardname .
FILTER CONTAINS(?awardname,"Nobel")
}
"""

print("Results")
run_query(queryString)

Results
[('award', 'http://www.wikidata.org/entity/Q35637'), ('awardname', 'Nobel Peace Prize')]
[('award', 'http://www.wikidata.org/entity/Q37922'), ('awardname', 'Nobel Prize in Literature')]
[('award', 'http://www.wikidata.org/entity/Q80061'), ('awardname', 'Nobel Prize in Physiology or Medicine')]
[('award', 'http://www.wikidata.org/entity/Q38104'), ('awardname', 'Nobel Prize in Physics')]


4

#### Found now i select only Italians authors

In [36]:
queryString = """
SELECT DISTINCT ?writername WHERE { 

?writer wdt:P27 ?country;
        wdt:P106 wd:Q36180;
        wdt:P166 wd:Q37922.

?writer <http://schema.org/name> ?writername .
FILTER (?country=wd:Q38 || ?country=wd:Q172579)
}
"""

print("Results")
run_query(queryString)

Results
[('writername', 'Grazia Deledda')]
[('writername', 'Giosuè Carducci')]
[('writername', 'Luigi Pirandello')]
[('writername', 'Dario Fo')]
[('writername', 'Salvatore Quasimodo')]


5

#### I count them

In [40]:
queryString = """
SELECT DISTINCT (COUNT (DISTINCT ?writer) AS ?number) WHERE { 

?writer wdt:P27 ?country;
        wdt:P166 wd:Q37922.

FILTER (?country=wd:Q38 || ?country=wd:Q172579)
}
"""

print("Results")
run_query(queryString)

Results
[('number', '6')]


1

## Task 5 - Are there books from Litarature Nobel Award winners which are not present in the Vatican Library? (if so, who is the author with more books not in the Vatical Library)?

#### I search for a property that connects the books of the Nobel winners and Vatican Library(wd:Q213678)

In [42]:
queryString = """
SELECT DISTINCT ?p ?pname WHERE { 

?writer wdt:P166 wd:Q37922;
        wdt:P800 ?work.
?work ?p wd:Q213678.

?p <http://schema.org/name> ?pname.

}
"""

print("Results")
run_query(queryString)

Results
Empty


0

#### Found nothing, let's try to inspect the propeterties

In [43]:
queryString = """
SELECT DISTINCT ?p ?pname WHERE { 

?writer wdt:P166 wd:Q37922;
        wdt:P800 ?work.
?work ?p ?obj.

?p <http://schema.org/name> ?pname.

FILTER CONTAINS(?pname,"Vatican")
}
"""

print("Results")
run_query(queryString)

Results
Empty


0

#### Still nothing, i search all incoming properties of the Vatican Library

In [44]:
queryString = """
SELECT DISTINCT ?p ?pname WHERE { 

?work ?p wd:Q213678.

?p <http://schema.org/name> ?pname.

}
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2389'), ('pname', 'organization directed by the office or position')]
[('p', 'http://www.wikidata.org/prop/direct/P9419'), ('pname', 'personal library at')]
[('p', 'http://www.wikidata.org/prop/direct/P108'), ('pname', 'employer')]
[('p', 'http://www.wikidata.org/prop/direct/P127'), ('pname', 'owned by')]
[('p', 'http://www.wikidata.org/prop/direct/P137'), ('pname', 'operator')]
[('p', 'http://www.wikidata.org/prop/direct/P1416'), ('pname', 'affiliation')]
[('p', 'http://www.wikidata.org/prop/direct/P189'), ('pname', 'location of discovery')]
[('p', 'http://www.wikidata.org/prop/direct/P195'), ('pname', 'collection')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('pname', 'location')]
[('p', 'http://www.wikidata.org/prop/direct/P301'), ('pname', "category's main topic")]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('pname', 'part of')]
[('p', 'http://www.wikidata.org/prop/direct/P485'), ('pname', 'archives at')]
[('p',

18

#### Try "archives at"(wd:P485)

In [47]:
queryString = """
SELECT DISTINCT ?work ?workname WHERE { 

?work wdt:P485 wd:Q213678.

?work <http://schema.org/name> ?workname.

}
"""

print("Results")
run_query(queryString)

Results
[('work', 'http://www.wikidata.org/entity/Q3057287'), ('workname', 'Ernst Steinmann')]


1

#### No...collection(wdt:195)

In [48]:
queryString = """
SELECT DISTINCT  ?workname WHERE { 

?work wdt:P195 wd:Q213678.

?work <http://schema.org/name> ?workname.

}
"""

print("Results")
run_query(queryString)

Results
[('workname', 'Bibbia di Federico da Montefeltro - BAV Urb.Lat.1&2')]
[('workname', 'Codex Vaticanus')]
[('workname', 'Lasimos Krater')]
[('workname', 'Bodmer Papyri')]
[('workname', 'Codex Vaticanus Graecus 64')]
[('workname', 'Barberini Gospels')]
[('workname', 'Divina Commedia de Federigo da Montefeltro - BAVaticana UrbLat365')]
[('workname', 'Book of Dede Korkut')]
[('workname', 'Rossiana Library')]
[('workname', 'Vergilius Vaticanus')]
[('workname', 'Minuscule 131')]
[('workname', 'Anjou Legendarium')]
[('workname', 'Codex Vaticanus 354')]
[('workname', 'Codex Assemanius')]
[('workname', 'Codex Ríos')]
[('workname', 'Joshua Roll')]
[('workname', 'breviary of  Matthias Corvin')]
[('workname', 'Andrew, the Apostle')]
[('workname', 'Apostolic Library Regina 1682')]
[('workname', 'Apostolic Library Reg. 1323')]
[('workname', 'Codex vaticanus latinus 3199')]
[('workname', 'Gregorian Sacramentary')]
[('workname', 'Codex Parisino-petropolitanus')]
[('workname', 'Chigi codex')]
[(

55

#### Count the books for each winner that are not in the Vatican Library

In [56]:
queryString = """
SELECT DISTINCT ?writername (COUNT(?work) AS ?number) WHERE { 

?writer wdt:P166 wd:Q37922;
        wdt:P800 ?work.
?work wdt:P31 ?type.


?writer <http://schema.org/name> ?writername.

FILTER ((?type=wd:Q47461344 || ?type=wd:Q7725634 || ?type=wd:Q571 || ?type=wd:Q8261) && (NOT EXISTS{?work wdt:P195 wd:Q213678.}))
}
GROUP BY(?writername)
ORDER BY DESC (?number)
LIMIT 1
"""

print("Results")
run_query(queryString)

Results
[('writername', 'Thomas Mann'), ('number', '54')]


1